In [1]:
from google.colab import drive
import warnings

drive.mount('/content/drive')
warnings.filterwarnings('ignore')

Mounted at /content/drive


In [2]:
import os
import glob
import pandas as pd

item_path = os.path.join('drive/MyDrive', 'tfg/data/*')
files = [file for file in glob.glob(item_path) if file.endswith('dataset.csv')]
item_name = max(files, key=os.path.getctime)

data = pd.read_csv(item_name)
data.head()

,log_id,log_msgname,log_wristopid,log_logtitle,log_lognotes,log_starttime,log_sampleinterval,log_duration,log_minalt,log_maxalt,...,sample_vent_var,sample_vo2_var,sample_corrected_variation,sample_ef_variation,sample_fom_variation,sample_hr_variation,sample_ibi_variation,sample_nrg_variation,sample_vent_variation,sample_vo2_variation
0,278217883,R6005A,0,1/9/2012 / 4:55:31 PM,NaN,09.01.2012 16:55:31,1,13802,0,0,...,1.037663,0.522519,2.264879e-16,0.176435,0.019404,0.019473,0.0,0.131683,0.081129,0.164662
1,188525594,R6005A,0,1/12/2012 / 4:51:31 PM,NaN,12.01.2012 16:51:31,1,11346,0,0,...,3.447306,0.717163,0.000000e+00,0.211841,0.149457,0.005515,0.0,0.135490,0.133375,0.173840
2,193452811,R6005A,0,4/2/2012 / 5:04:24 PM,NaN,02.04.2012 17:04:24,1,11367,0,0,...,83.731858,13.486994,0.000000e+00,1.592938,0.296382,0.167501,0.0,0.732945,0.824158,0.827888
3,126785920,R6005A,0,1/13/2012 / 4:57:24 PM,NaN,13.01.2012 16:57:24,1,15777,0,0,...,0.511565,0.316079,2.224877e-16,0.150455,0.007499,0.014293,0.0,0.111798,0.058817,0.133348
4,256856976,R6005A,0,4/10/2012 / 9:23:04 AM,NaN,10.04.2012 09:23:04,1,7986,0,0,...,37.459516,7.673069,0.000000e+00,0.725459,0.307949,0.130026,0.0,0.502513,0.500688,0.560591


In [3]:
data.shape

(5772, 248)

In [ ]:
def exclude_fields(data):
    rows, columns = data.shape
    print(f'Summary of dataframe ({rows}, {columns})')
    print('Field'.ljust(35), 'Sample value'.ljust(30), 'Null values'.ljust(20), 'Data type'.ljust(20), 'Unique values'.ljust(20), 'Excluded')
    excluded = list()
    exclude = False
    for field, value in data.iloc[0].items():
        try:
            value = round(value, 4)
        except:
            pass
        dtype = str(type(value))[8:-2]
        if dtype.startswith('numpy'):
            dtype = dtype[6:]
        if dtype.endswith('64'): 
            dtype = dtype[:-2]
        elif dtype.endswith('_'):
            dtype = dtype[:-1]
        null = pd.isnull(data[field]).mean()
        unique = len(pd.unique(data[field]))
        if null == 1.0 or unique == 1 or field.endswith('_id') or field.endswith('title') or field in {'log_starttime', 'previous_ergon', 'previous_fergo', 'next_ergon', 'next_fergo'}:
            exclude = True
            excluded.append(field)
        print(field.ljust(35), str(value).ljust(30), str(null).ljust(20), dtype.ljust(20), str(unique).ljust(20), exclude)
        exclude = False
    print(f'Total excluded fields: {len(excluded)}')
    data_excluded = data.drop(excluded, axis=1)
    return data_excluded

data_excluded = exclude_fields(data)

Summary of dataframe (5772, 248)
Field                               Sample value                   Null values          Data type            Unique values        Excluded
log_id                              278217883                      0.0                  int                  5772                 True
log_msgname                         R6005A                         0.0                  str                  1                    True
log_wristopid                       0                              0.0                  int                  1                    True
log_logtitle                        1/9/2012 / 4:55:31 PM          0.0                  str                  1636                 True
log_lognotes                        nan                            1.0                  float                1                    True
log_starttime                       09.01.2012 16:55:31            0.0                  str                  1636                 True
log_sampleinterval

In [ ]:
def convert_fields(data):
    print('Converting user date of birth field into year of birth field')
    data_converted = data.drop('user_fn', axis=1)
    data_converted['user_year_of_birth'] = data['user_fn'].apply(lambda date: int(date[:4]))
    # formats = {'log_starttime': '%d.%m.%Y %X', 'user_fn': '%Y-%m-%d', 'previous_fergo': '%Y-%m-%d', 'next_fergo': '%Y-%m-%d'}
    # print(f'Date/time fields to convert: {", ".join(key for key in formats.keys())}')
    # print('Before'.ljust(41), 'After')
    # for field, format in formats.items():
    #     data_converted[field] = pd.to_datetime(data[field], format=format)
    # print('Sample value'.ljust(25), 'Data type'.ljust(15), 'Sample value'.ljust(25), 'Data type')
    # for field in formats.keys():
    #     previous_value = data[field]
    #     next_value = data_converted[field]
    #     print(previous_value[0].ljust(25), str(type(previous_value[0]))[-5:-2].ljust(15), str(next_value[0]).ljust(25), str(type(next_value[0]))[-11:-2].lower())
    # print()

    cancers = list({cancer for pair in pd.unique(pd.concat([data['previous_cancer'], data['next_cancer']])) for cancer in pair.split('/')})
    print(f'Cancer categories: {", ".join([cancer.lower() for cancer in cancers])}')
    for field in ('previous_cancer', 'next_cancer'):
        for cancer in cancers:
            data_converted[f'{field}_{cancer.lower()}'] = data[field].apply(lambda value: int(cancer in value))
        data_converted.drop(field, axis=1, inplace=True)
    return data_converted

data_converted = convert_fields(data_excluded)

Converting user date of birth field into year of birth field
Cancer categories: mama, laringe, prostata, pancreas, utero, vejiga, gastrico, linfoma, colon, tiroides


In [ ]:
item_name_processed = f'{item_name[:-12]}-processed.csv'
data_converted.to_csv(item_name_processed, index=False)